In [3]:
# Install libraries if needed
!pip install requests pandas --quiet

import requests
import pandas as pd
from functools import reduce

# ✅ Country - Only Vietnam
country = "VNM"

# ✅ Indicators required (Updated with IIP)
indicators = {
    "GDP Growth (%)": "NY.GDP.MKTP.KD.ZG",
    "CPI (Inflation, annual %)": "FP.CPI.TOTL.ZG",
    "Exchange Rate (USD/VND)": "PA.NUS.FCRF",
    "Interest Rate (%) (Lending Rate Proxy)": "FR.INR.LEND",
    "FDI Inflow (US$)": "BX.KLT.DINV.CD.WD",
    "Industrial Production Growth (IIP %)": "IP.PRD.INDX.ZG"
}

start_year = 2010
end_year = 2024

# ✅ World Bank API data fetch function
def get_wb_data(country_code, indicator_code, start, end):
    url = (
        f"https://api.worldbank.org/v2/country/{country_code}/indicator/"
        f"{indicator_code}?format=json&date={start}:{end}&per_page=2000"
    )
    r = requests.get(url)
    data = r.json()

    # Nếu không có dữ liệu
    if len(data) < 2 or not isinstance(data[1], list) or len(data[1]) == 0:
        print(f"⚠️ Warning: No data for indicator {indicator_code}")
        return pd.DataFrame()

    df = pd.DataFrame(data[1])[["date", "value"]]
    df["Year"] = df["date"].astype(int)
    df.drop(columns=["date"], inplace=True)
    df.rename(columns={"value": indicator_code}, inplace=True)
    return df

# ✅ Retrieve all indicators for Vietnam
data_frames = []
for name, code in indicators.items():
    print(f"Fetching {name} ...")
    df_temp = get_wb_data(country, code, start_year, end_year)
    if df_temp.empty:
        continue  # bỏ DataFrame rỗng
    df_temp.rename(columns={code: name}, inplace=True)
    data_frames.append(df_temp)

# ✅ Merge into a single table, chỉ khi có DataFrame tồn tại
if data_frames:
    df = reduce(lambda left, right: pd.merge(left, right, on="Year", how="outer"), data_frames)
    df = df.sort_values("Year").reset_index(drop=True)
else:
    df = pd.DataFrame()
    print("❌ No data available for any indicator!")

# ✅ Formatting display
pd.options.display.float_format = '{:,.2f}'.format

# ✅ Export Excel nếu có dữ liệu
if not df.empty:
    output_filename = "Vietnam_Macro_Indicators_2010_2024.xlsx"
    df.to_excel(output_filename, index=False)
    print("✅ File exported successfully:", output_filename)
else:
    print("❌ No Excel file exported, data empty.")

df


Fetching GDP Growth (%) ...
Fetching CPI (Inflation, annual %) ...
Fetching Exchange Rate (USD/VND) ...
Fetching Interest Rate (%) (Lending Rate Proxy) ...
Fetching FDI Inflow (US$) ...
Fetching Industrial Production Growth (IIP %) ...
⚠️ Warning: No data for indicator IP.PRD.INDX.ZG
✅ File exported successfully: Vietnam_Macro_Indicators_2010_2024.xlsx


,GDP Growth (%),Year,"CPI (Inflation, annual %)",Exchange Rate (USD/VND),Interest Rate (%) (Lending Rate Proxy),FDI Inflow (US$)
0,6.42,2010,9.21,"18,612.92",13.14,8000000000
1,6.41,2011,18.68,"20,509.75",16.95,7430000000
2,5.50,2012,9.09,"20,828.00",13.47,8368000000
3,5.55,2013,6.59,"20,933.42",10.37,8900000000
4,6.42,2014,4.08,"21,148.00",8.66,9200000000
5,6.99,2015,0.63,"21,697.57",7.12,11800000000
6,6.69,2016,2.67,"21,935.00",6.96,12600000000
7,6.94,2017,3.52,"22,370.09",7.07,14100000000
8,7.47,2018,3.54,"22,602.05",7.37,15500000000
9,7.36,2019,2.80,"23,050.24",7.71,16120000000
